# team myzw
### ToDo
逆強化学習  
相対概念  
セマンティックマッピング   
シグバース  
能動的地図学習    

TODO
* 特徴量抽出
 * wordコードブック作成
　* vision
 * motion
* mMLDA学習
* BHMM学習
* HMM学習[学習＋プランニング]
* 言語からZの推定

### 岩田さん　データ形成

In [1]:
iwata_data_path = "/home/robocup/Desktop/miyazawa_crest_2018/caption/"

In [2]:
try:
    os.mkdir(os.path.join(iwata_data_path,"RecogData"))
    os.mkdir(os.path.join(iwata_data_path,"RecogData/words"))
except:
    pass

In [3]:
ls /home/robocup/Desktop/miyazawa_crest_2018/caption/

RecogData/        exp_environment_1/  exp_environment_3/  exp_environment_5/
RecogData.tar.gz  exp_environment_2/  exp_environment_4/  exp_environment_6/


In [4]:
import pandas as pd
import numpy as np
import MeCab
from word_client import WordClient
word_client = WordClient()
import glob
import os

In [5]:
place_list = ["sinnsitu","kittinn","huro","doa","shosai"]
objects_list = ["door","table","doll","huru-tu","drink"]
class_nums = [3,3,3,3,3]
START_CLASS = 3
END_CLASS = 4

In [89]:
place = pd.read_csv("/home/robocup/Desktop/miyazawa_crest_2018/caption/exp_environment_1/index000.txt",header=None)

In [105]:
task_list = []
action_list = []
caption_list = []
objects_list = []
place_list = []
correct_object = []
correct_motion = []
scene_id = 0
vision_eye = np.eye(len(class_nums))

for task_id in range(1,7):
    action_num = len(glob.glob(os.path.join(iwata_data_path,"exp_environment_{0}".format(task_id),"action*")))
    for action_id in range(action_num):
        caption_path = os.path.join(iwata_data_path,"exp_environment_{0}/caption{1:03d}.txt".format(task_id,action_id))
        motions_path = os.path.join(iwata_data_path,"exp_environment_{0}/action{1:03d}.txt".format(task_id,action_id))
        place_path = os.path.join(iwata_data_path,"exp_environment_{0}/index{1:03d}.txt".format(task_id,action_id))
        motions = np.loadtxt(motions_path, delimiter=",", skiprows=1,dtype=np.int32)
        object_id = np.loadtxt(motions_path, delimiter=",", usecols=0,dtype=np.int32)[0]
        place = pd.read_csv(place_path,header=None)
        
        caption = pd.read_csv(caption_path,header=None)
        caption.columns = ["sentences"]
        caption["secne_id"] = scene_id
        caption_list.append(caption)
        objects_list.append(vision_eye[object_id])
        
        place_hist = np.zeros(100)
        for i,idx in enumerate(place.iloc[:,0].value_counts().index):
            place_hist[idx] = place.iloc[:,0].value_counts().values[i]
        place_list.append(place_hist)
        
        correct_object.append(object_id)
        correct_motion.append(object_id)
        
        motions = np.insert(motions,0,[START_CLASS,END_CLASS])
        motions = motions +  object_id*7
        tmp_motion_hists = []
        for i in range(7*len(class_nums)):
            tmp_motion_hists.append(len(np.where(motions==i)[0]))
        action_list.append(tmp_motion_hists)
        
        scene_id+= 1

In [107]:
np.savetxt(os.path.join(iwata_data_path,"RecogData/motion.Motion"),np.array(action_list),fmt="%d")
np.savetxt(os.path.join(iwata_data_path,"RecogData/vision.Object"),np.array(objects_list),fmt="%d")
np.savetxt(os.path.join(iwata_data_path,"RecogData/place.Place"),np.array(place_list),fmt="%d")

np.savetxt(os.path.join(iwata_data_path,"RecogData/CorrectObject.txt"),np.array(correct_object),fmt="%d")
np.savetxt(os.path.join(iwata_data_path,"RecogData/CorrectMotion.txt"),np.array(correct_motion),fmt="%d")

In [8]:
caption_pd = pd.concat(caption_list,axis=0)

In [9]:
caption_pd

,sentences,secne_id
0,コア寝室だよ,0
1,寝室で飲み物をつかむ,0
2,寝室で飲み物を把持する,0
3,寝室で飲み物に手を伸ばす,0
4,寝室で飲み物をつかむ,0
5,寝室で飲み物を掴んだ,0
0,ココアドア前だよ,1
1,ドア前でドアを開ける,1
2,ドア前でドアを押す,1
3,ドア前でドアを開く,1


In [10]:
caption_pd.sentences.to_csv(os.path.join(iwata_data_path,"RecogData/sentences.txt"),index=False)

In [11]:
for i in range(1,17):
    word_dir = os.path.join(iwata_data_path,"RecogData/words/{0:03d}".format(i))
    try:
        os.mkdir(word_dir)
    except:
        pass
    
    word_client.update(word_dir)
    word_client.setAccept()
    # word の処理
    for sen in caption_pd[caption_pd.secne_id==i].sentences.values:
        word_client.sentence_data_cb(sen)
    word_client.setReject()
    word_client.split_sentences(word_dir)
    word_client.sumalize_utter(word_dir)
    sentences_file_path = os.path.join(word_dir,"sentences.txt")
    word_client.update_word_codebook(sentences_file_path)
    word_client.words2histogram(
    os.path.join(word_dir,"sentences.txt"),
    os.path.join(word_dir,"hist.txt")
    )
    word_client.dump_codebook(os.path.join(word_dir,"codebook.txt"))

ドア 前 で ドア を 開ける 

ドア 前 で ドア を 押す 

ココア ドア 前 だ よ 

ドア 前 で ドア を 開く 

ドア から 入る 

ドア 前 で ドア を 開い た 

ドア から 入る 

ドア 前 で ドア を 開ける 

ドア 前 で ドア を 開い た 

ドア 前 で ドア を 開く 

ココア ドア 前 だ よ 

ドア 前 で ドア を 押す 

キッチン 

キッチン で 物 を 設置 する 

キッチン で 物 を 置く 

キッチン で 物 を 置く 

キッチン で 物 を 置い た 

キッチン 

これ が キッチン だ よ 

キッチン で 物 を 置い た 

キッチン 

キッチン 

キッチン で 物 を 置く 

これ が キッチン だ よ 

キッチン で 物 を 置く 

キッチン で 物 を 設置 する 

ココア ドア 前 だ よ 

寝室 から 出る 

風呂 で 飲み物 を 置く 

ココア 風呂 だ よ 

寝室 で 飲み物 を つかむ 

風呂 で 飲み物 を 置い た 

寝室 で 飲み物 を 味 する 

風呂 に 着い た 

風呂 です 

これ が 真実 です 

風呂 で 飲み物 を 設置 し た 

親子 で 飲み物 を 掴ん だ 

風呂 で 物 を 置く 

寝室 で 飲み物 に 手 を 伸ばし た 

寝室 で 飲み物 を はじ し た 

ドア 前 です 

風呂 で 飲み物 を 設置 する 

ドア 前 です 

風呂 で 飲み物 を 設置 し た 

風呂 で 物 を 置く 

風呂 です 

親子 で 飲み物 を 掴ん だ 

風呂 に 着い た 

寝室 で 飲み物 を つかむ 

ココア ドア 前 だ よ 

風呂 で 飲み物 を 置い た 

ココア 風呂 だ よ 

寝室 で 飲み物 を はじ し た 

寝室 で 飲み物 に 手 を 伸ばし た 

寝室 で 飲み物 を 味 する 

風呂 で 飲み物 を 置く 

寝室 から 出る 

これ が 真実 です 

風呂 で 飲み物 を 設置 する 

風呂 で 飲み物 を 置い た 

風呂 で 飲み物 を 設置 し た 

風呂 で 飲み物 を 設置 する 

風呂 で 飲み物 を 置い た 

風呂 

In [12]:
words_hists = []
for i in range(1,17):
    word_hist_path = os.path.join(iwata_data_path,"RecogData/words/{0:03d}/hist.txt".format(i))
    words_hists.append(np.loadtxt(word_hist_path))

In [13]:
np.savetxt(os.path.join(iwata_data_path,"RecogData/words.All"),np.array(words_hists),fmt="%d")

In [109]:
action_num_list = []
for task_id in range(1,17):
    action_num = len(glob.glob(os.path.join(iwata_data_path,"exp_environment_{0}".format(task_id),"action*")))
    action_num_list.append(action_num)

In [110]:
np.savetxt("/home/robocup/Desktop/miyazawa_crest_2018/caption/action_nums.txt",action_num_list,fmt="%d")

In [16]:
all_sentence = []
for i in range(1,17):
    word_hist_path = os.path.join(iwata_data_path,"RecogData/words/{0:03d}/sentences.txt".format(i))
    tmp_sen_list = np.loadtxt(word_hist_path,dtype=str,delimiter="\n").tolist()
    if type(tmp_sen_list) is str:
        tmp_sen_list = [tmp_sen_list]
    all_sentence.extend(tmp_sen_list)
sentences_path = os.path.join(iwata_data_path,"RecogData/sentences.txt")
np.savetxt(sentences_path,all_sentence,fmt="%s")

/home/robocup/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: loadtxt: Empty input file: "/home/robocup/Desktop/miyazawa_crest_2018/caption/RecogData/words/016/sentences.txt"


In [17]:
print all_sentence[0]

ドア から 入る 


進捗  
ドアを作っているところ  
物体の情報を

jointトラジェクトリー

場所概念の可視化

場所概念
既知概念環境で

GP-HSMM:rosbagで学習できる
IK:シグバース
学習データ：


場所概念の学習時間：　

### 場所概念の正解を与える

In [29]:
import IPython

In [36]:
place_correct = []
exp_dirs = glob.glob(os.path.join(iwata_data_path,"exp_environment*"))
exp_dirs.sort()
for exp in exp_dirs:
    captions = glob.glob(os.path.join(exp,"caption*"))
    captions.sort()
    for caption in captions:
        tmp = np.loadtxt(caption,dtype=str)
        try:
            for i in tmp:
                print i
        except:
            print tmp
        print ""
        print caption
        print ""
        print "寝室：    0"
        print "キッチン: 1"
        print "風呂:     2"
        print "ドア:     3"
        print "書斎:     4"
        
        place_correct.append(int(raw_input("place_id")))
        
        IPython.display.clear_output()

In [74]:
place_eye = np.eye(4)
place_hist = []
for place in place_correct:
    place_hist.append(place_eye[place])
np.savetxt(os.path.join(iwata_data_path,"RecogData/CorrectPlace.txt"),place_correct,fmt="%d")

np.savetxt(os.path.join(iwata_data_path,"RecogData/place.Place"),place_hist,fmt="%d")